In [109]:
import pandas as pd
import re

In [110]:
# Leer el archivo .sql
with open('Query_polizas_gs.sql', 'r', encoding='utf-8') as file:
    sql_query = file.read()

In [111]:
# Define una expresión regular para encontrar todo lo que está entre '--' y '---'
patron_1 = r'--Sumery:(.*?)-----------------------------------------------------------------------------------------------------------------------------'

patron_2 = r'(Step|LC)(.*?)--'

patron_3 = r'--(.*?)--'

# Encuentra todas las coincidencias en el contenido del archivo
coincidencias = re.findall(patron_2, sql_query, re.DOTALL)

# Imprime las coincidencias encontradas
for coincidencia in coincidencias:
    print(coincidencia)

('Step', ' 1: Se obtienen todas las talta de HSBC a excepción de las que provienen de campañas')
('LC', ': Se limitan las altas por id de estructura de gestion')
('LC', ': Se limita el universo a la primera póliza (certificado)')
('Step', ' 2: Proceso para obtener las altas de HSBC que se gestionana a travez de campañas')
('Step', ' 2_1: Se genera una temporal con las solicitudes generadas en las campañas de HSBC')
('LC', ': Se limita el universo solo a las campañas de HSBC')
('LC', ': Trae solamente las gestiones de ventas')
('Step', ' 2_2: Cruce con cotizaciones para obtener detalles de la solicitud convertida en póliza')
('LC', ': Generarmo un flg para identifiar aquellas pólizas provenientes de campañas')
('Step', ' 2_3: Se realiza el cruce con certificados utilizando num_solicitud de campañas y NPOLICY de Certificados para traer las dimensiones de ncertipol, id_producto, id_branch')
('LC', ': Generamos un flg para identifiar aquellas pólizas provenientes de campañas')
('Step', ' 2

In [112]:
#print(sql_query)